In [ ]:
import os
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# matplotlib.use("agg")

from exp_spec_info import *
from plot_info import *
from select_data import *

In [ ]:
# Processed median path
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

# Matrix data
small_matrix_data_path = "C:\\Users\\dosre\\dev\\precision-cascade\\scripts\\python\\view_experimentation\\data\\small-experimental-matrix-data.csv"
large_matrix_data_path = "C:\\Users\\dosre\\dev\\precision-cascade\\scripts\\python\\view_experimentation\\data\\large-experimental-matrix-data.csv"

# Plot output directory
plots_6_dir = "C:\\Users\\dosre\\Desktop\\MSCthesis\\thesis\\body_chapters\\chap_6\\images\\precond-analysis"
os.makedirs(plots_6_dir, exist_ok=True)

In [ ]:
# Load matrix data
small_matrix_data = pd.read_csv(small_matrix_data_path).rename({"name": "matrix"}, axis=1)
large_matrix_data = pd.read_csv(large_matrix_data_path).rename({"name": "matrix"}, axis=1)
all_matrix_data = pd.concat([small_matrix_data.drop("cond", axis=1), large_matrix_data])

# Load data and filter for only gmres m solvers
median_data = pd.read_pickle(median_data_path)
precond_median_data = median_data[
    (median_data["solver"].transform(lambda x: (x in PC_SOLVERS))) &
    median_data["setup"].isin({"ilu0", "ilutp1em2", "ilutp1em4"})
]

##### Prune Non-Complaint Preconditioned Experiments

In [ ]:
frac_err_compliance_threshold = 1.
precond_not_failed = (precond_median_data["med_rel_res"]/(precond_median_data["med_rel_res_frac_err_fp64"]+1) <= 1)
n_precon_failed = (~precond_not_failed).sum()
keep = (
    (precond_median_data["med_rel_res_frac_err_fp64"] <= frac_err_compliance_threshold) &
    precond_not_failed # FP64 did not get worse
)
compliant_data = precond_median_data[keep]
eliminated_data = precond_median_data[~keep]
mat_index_dict = dict(zip(
    eliminated_data["matrix"].unique(),
    np.arange(0, len(eliminated_data["matrix"].unique()))
))
index_mat_dict = dict(zip(
    np.arange(0, len(eliminated_data["matrix"].unique())),
    eliminated_data["matrix"].unique()
))

print(f"Valid data percentage: {len(compliant_data)/len(precond_median_data):.2g}")
print(f"N Precond Failed: {n_precon_failed/len(precond_median_data):.2g}")
print(f"Eliminated data breakdown:")
print(eliminated_data["setup"].value_counts())
print(eliminated_data["restart_param"].value_counts())
print(eliminated_data["solver"].value_counts())
print(eliminated_data["matrix"].value_counts())

fig, axs = plt.subplots(1, 2, figsize=(6.4, 4.8), width_ratios=[0.75, 0.25])

ax, ax_legend = axs

for solver in PC_SOLVERS:
    ax.scatter(
        eliminated_data["med_rel_res_frac_err_fp64"][eliminated_data["solver"] == solver],
        eliminated_data["matrix"][eliminated_data["solver"] == solver].transform(
            lambda mat: mat_index_dict[mat]
        ),
        eliminated_data["restart_param"][eliminated_data["solver"] == solver],
        marker=".",
        color=SOLVER_CLR_DICT[solver],
        label=solver
    )

ax.set_yticks(np.arange(0, len(eliminated_data["matrix"].unique())))
ax.set_yticklabels([index_mat_dict[int(label)] for label in np.arange(0, len(eliminated_data["matrix"].unique()))])
ax.semilogx()
ax.grid()
ax.set_axisbelow(True)
ax.set_ylabel("Linear Solve Matrix")
ax.set_xlabel("Med. Rel. Res. Frac. Error to FP64")
ax.set_title(f"Non-Compliant Experiments with Rel. Res. Frac. Error Threshold $(>{frac_err_compliance_threshold})$")

ax_legend.legend(
    handles=[matplotlib.lines.Line2D([0], [0], marker=".", linestyle="None", color=SOLVER_CLR_DICT[solver]) for solver in PC_SOLVERS],
    labels=PC_SOLVERS
)
ax_legend.tick_params(
    axis="both", bottom=False, left=False, labelbottom=False, labelleft=False
)
ax_legend.axis("off")


plt.savefig(
    os.path.join(plots_6_dir, "precond-threshold-non-compliance.png"),
    bbox_inches="tight"
)
plt.show()



##### 6.4 Precond FP-GMRES and PC-GMRES Relative Time Comparison

In [ ]:
n_compliant_matrices = len(compliant_data["matrix"].unique())
mat_index_dict = dict(zip(
    compliant_data["matrix"].unique(), np.arange(0, n_compliant_matrices)
))
index_mat_dict = dict(zip(
    np.arange(0, n_compliant_matrices), compliant_data["matrix"].unique()
))

comparison_data = []

for restart_param in RESTART_PARAMS:

    restart_data = compliant_data[compliant_data["restart_param"] == restart_param]

    data_point = "med_rel_time_fp64"

    ilu0_data = restart_data[restart_data["setup"] == "ilu0"]
    ilutp1em2_data = restart_data[restart_data["setup"] == "ilutp1em2"]
    ilutp1em4_data = restart_data[restart_data["setup"] == "ilutp1em4"]

    fig, axs = plt.subplots(
        1, len(PC_SOLVERS), figsize=(11.5, 8), sharey=True
    )
    x_range = [0., 100.]

    for ax, solver in zip(axs, PC_SOLVERS):
        for data, setup, marker, line, color in [
            (ilu0_data, "ilu0", ".", "--", SOLVER_CLR_DICT[solver]),
            (ilutp1em2_data, "ilutp1em2", "x", "-.", SOLVER_CLR_DICT[solver]),
            (ilutp1em4_data, "ilutp1em4", "v", ":", SOLVER_CLR_DICT[solver])
        ]:

            solver_data = data[data["solver"] == solver]

            ax.scatter(
                solver_data[data_point],
                solver_data["matrix"].transform(lambda mat: mat_index_dict[mat]),
                alpha=1.,
                color=color,
                zorder=2,
                marker=marker
            )
            ax.axvline(
                1.,
                color=SOLVER_CLR_DICT["FP FP64"],
                linestyle="solid",
                zorder=1
            )
            ax.axvline(
                solver_data[data_point].median(),
                color=color,
                linestyle=line,
                zorder=2
            )
            ax.set_title(f"{solver}")

            ax.set_xlim(x_range[0], x_range[1])

            comparison_data.append(
                {
                    "setup": setup,
                    "restart_param": restart_param,
                    "solver": solver,
                    data_point: solver_data[data_point].median()
                }
            )

        off_screen_data = pd.concat(
            [ilu0_data[ilu0_data[data_point] >= x_range[1]]["matrix"],
             ilutp1em2_data[ilutp1em2_data[data_point] >= x_range[1]]["matrix"],
             ilutp1em4_data[ilutp1em4_data[data_point] >= x_range[1]]["matrix"]]
        ).value_counts()
        for matrix in off_screen_data.index:
            ax.text(
                x_range[0]+0.99*(x_range[1]-x_range[0]),
                mat_index_dict[matrix],
                f"{off_screen_data.loc[matrix]}$\\rightarrow$",
                verticalalignment='center',
                horizontalalignment='right'
            )

    for ax in axs:
        ax.grid()
    axs[3].set_xlabel(f"Median Time Relative to FP64")
    axs[0].set_yticks(np.arange(0, n_compliant_matrices))
    axs[0].set_yticklabels([index_mat_dict[label] for label in np.arange(0, n_compliant_matrices)])
    axs[0].set_ylabel("Linear Solve Matrix")

    fig.suptitle(f"Preconditioned PC-GMRES(${restart_param}$) Time Relative to FP64")
    fig.tight_layout()

    plt.savefig(
        os.path.join(plots_6_dir, f"{restart_param:03d}-precond-rel-time.png")
    )
    plt.show()
    plt.close()

comparison_data = pd.DataFrame(comparison_data)


In [ ]:
ilu0 = comparison_data[comparison_data["setup"] == "ilu0"]
ilu0 = ilu0.drop("setup", axis=1)
ilu0 = ilu0.pivot(index="solver", columns="restart_param", values="med_rel_time_fp64").loc[PC_SOLVERS[::-1]]
ilu0 = ilu0.round(2)

ilutp1em2 = comparison_data[comparison_data["setup"] == "ilutp1em2"]
ilutp1em2 = ilutp1em2.drop("setup", axis=1)
ilutp1em2 = ilutp1em2.pivot(index="solver", columns="restart_param", values="med_rel_time_fp64").loc[PC_SOLVERS[::-1]]
ilutp1em2 = ilutp1em2.round(2)

ilutp1em4 = comparison_data[comparison_data["setup"] == "ilutp1em4"]
ilutp1em4 = ilutp1em4.drop("setup", axis=1)
ilutp1em4 = ilutp1em4.pivot(index="solver", columns="restart_param", values="med_rel_time_fp64").loc[PC_SOLVERS[::-1]]
ilutp1em4 = ilutp1em4.round(2)

max_val = 1.02
min_val = 0.77

fig, ax = plt.subplots()

solvers = ilu0.index.to_numpy()
restart_params = ilu0.columns.to_numpy()

X, Y = np.meshgrid(np.arange(0, len(restart_params)), np.arange(0, len(solvers)))
ax.pcolormesh(X, Y, ilu0.to_numpy(), cmap="RdYlGn_r", vmin=min_val, vmax=max_val)
for i in np.arange(0, len(restart_params)):
    for j in np.arange(0, len(solvers)):
        ax.text(i, j, ilu0.loc[solvers[j]][restart_params[i]], ha="center", va="center", fontsize=9)

ax.set_xticks(np.arange(0, len(restart_params)))
ax.set_xticklabels([restart_params[i] for i in np.arange(0, len(restart_params))])
ax.set_xlabel("Restart Parameter")
ax.set_yticks(np.arange(0, len(solvers)))
ax.set_yticklabels([solvers[i] for i in np.arange(0, len(solvers))])
ax.set_ylabel("PC-GMRES($m$) Solver")
ax.set_title(f"{SETUP_NAME_MAPPING["ilu0"]} PC-GMRES($m$) Median Relative Time to FP64")

plt.savefig(
    os.path.join(plots_6_dir, "rel-time-summary-ilu0.png"),
    bbox_inches="tight"
)
plt.show()

fig, ax = plt.subplots()

solvers = ilutp1em2.index.to_numpy()
restart_params = ilutp1em2.columns.to_numpy()

X, Y = np.meshgrid(np.arange(0, len(restart_params)), np.arange(0, len(solvers)))
ax.pcolormesh(X, Y, ilutp1em2.to_numpy(), cmap="RdYlGn_r", vmin=min_val, vmax=max_val)
for i in np.arange(0, len(restart_params)):
    for j in np.arange(0, len(solvers)):
        ax.text(i, j, ilutp1em2.loc[solvers[j]][restart_params[i]], ha="center", va="center", fontsize=9)

ax.set_xticks(np.arange(0, len(restart_params)))
ax.set_xticklabels([restart_params[i] for i in np.arange(0, len(restart_params))])
ax.set_xlabel("Restart Parameter")
ax.set_yticks(np.arange(0, len(solvers)))
ax.set_yticklabels([solvers[i] for i in np.arange(0, len(solvers))])
ax.set_ylabel("PC-GMRES($m$) Solver")
ax.set_title(f"{SETUP_NAME_MAPPING["ilutp1em2"]} PC-GMRES($m$) Median Relative Time to FP64")

plt.savefig(
    os.path.join(plots_6_dir, "rel-time-summary-ilutp1em2.png"),
    bbox_inches="tight"
)
plt.show()

fig, ax = plt.subplots()

solvers = ilutp1em4.index.to_numpy()
restart_params = ilutp1em4.columns.to_numpy()

X, Y = np.meshgrid(np.arange(0, len(restart_params)), np.arange(0, len(solvers)))
ax.pcolormesh(X, Y, ilutp1em4.to_numpy(), cmap="RdYlGn_r", vmin=min_val, vmax=max_val)
for i in np.arange(0, len(restart_params)):
    for j in np.arange(0, len(solvers)):
        ax.text(i, j, ilutp1em4.loc[solvers[j]][restart_params[i]], ha="center", va="center", fontsize=9)

ax.set_xticks(np.arange(0, len(restart_params)))
ax.set_xticklabels([restart_params[i] for i in np.arange(0, len(restart_params))])
ax.set_xlabel("Restart Parameter")
ax.set_yticks(np.arange(0, len(solvers)))
ax.set_yticklabels([solvers[i] for i in np.arange(0, len(solvers))])
ax.set_ylabel("PC-GMRES($m$) Solver")
ax.set_title(f"{SETUP_NAME_MAPPING["ilutp1em4"]} PC-GMRES($m$) Median Time Relative to FP64")

plt.savefig(
    os.path.join(plots_6_dir, "rel-time-summary-ilutp1em4.png"),
    bbox_inches="tight"
)
plt.show()